# Quantum Oracle Demo

## Build and Compile the Circuit

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
q = QuantumRegister(4, 'q')
tmp = QuantumRegister(1, 'tmp')
res = ClassicalRegister(4, 'res')

In [ ]:
# Build oracle. The secret number is represented by integer s.
s = 0b1001
oracle = QuantumCircuit(q, tmp)
for i in range(len(q)):
    if (s & (1 << i)):
        oracle.cx(q[i], tmp[0])

In [ ]:
# Embed oracle in the larger BV circuit
bv = QuantumCircuit(q, tmp, res)
bv.x(tmp[0])
bv.barrier()
bv.h(q)
bv.h(tmp)
bv.barrier()
bv += oracle
bv.barrier()
bv.h(q)
bv.h(tmp)
bv.barrier()
bv.measure(q, res);

In [ ]:
bv.draw(output='latex', scale=0.7)

## Execute on a Simulator

In [ ]:
# Access local simulators from Qiskit Aer
from qiskit import Aer, execute
Aer.backends()

In [ ]:
simulator = Aer.get_backend('qasm_simulator')
sim_result = execute(bv, simulator).result()

In [ ]:
from qiskit.tools.visualization import plot_histogram
sim_counts = sim_result.get_counts()
sim_counts

In [ ]:
plot_histogram(sim_counts)

The Bernstein-Vazirani circuit finds the secret bitstring 100% of the time, with just 1 Oracle query.

## Execute on a Quantum Computer

In [ ]:
# authenticate to use IBMQ remote backends 
from qiskit import IBMQ
IBMQ.load_accounts()
IBMQ.backends()

In [ ]:
# import jupyter magic tools to ease tracking of submitted job's progress
from qiskit.tools.jupyter import *

In [ ]:
%qiskit_backend_overview

In [ ]:
# let's choose a 5-qubit device and execute
device = IBMQ.get_backend('ibmqx4')

In [ ]:
%%qiskit_job_status

# qiskit jobs are submitted asynchronously. You can monitor a submitted job's status
job = execute(bv, device, shots=1024)

In [ ]:
# NOTICE! Executing this cell will block until result is available
device_result = job.result()
device_counts = device_result.get_counts()
plot_histogram(device_counts)

Real devices are noisy. We repeat the experiment for 1000 shots to find the correct answer among the noise.